In [ ]:
from matplotlib import pyplot as plt
import tensorflow_datasets
import tensorflow as tf
import numpy as np

In [ ]:
def normilize(image, label):
    return tf.cast(image, tf.float32) / 256, label

In [ ]:
(train, test), info = tensorflow_datasets.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

In [ ]:
info.splits

In [ ]:
train = train.map(normilize)
train = train.cache()
train = train.shuffle(info.splits['train'].num_examples)
train = train.batch(256)
train = train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
test = test.map(normilize)
test = test.cache()
test = test.batch(256)
test = test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
history = model.fit(train, epochs=10, validation_data=test)
model.save(f"model-tf.h5")

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'], color='green', label='Train')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='red', label='Test')
plt.title('Accuracy')
plt.legend()
plt.show()